In [ ]:
# @title Google Drive Setup
from google.colab import drive

# Mount Google Drive to access files
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# @title Import Libraries
import cv2
import os
import numpy as np
from skimage.metrics import structural_similarity as ssim
import matplotlib.pyplot as plt
from scipy.stats import entropy as scipy_entropy
from PIL import Image
import numpy as np
from skimage.filters.rank import entropy as skimage_entropy
from skimage.morphology import disk
from skimage import img_as_ubyte


In [ ]:
# @title Entropy Calculation Function
def calculate_entropy(image):
    # Flatten the image array to 1D
    image_flat = image.flatten()

    # Count frequency of pixel values
    histogram, _ = np.histogram(image_flat, bins=256, range=(0, 256), density=True)

    # Calculate entropy
    return scipy_entropy(histogram)


In [ ]:
# @title Load Images
image_folder = "/content/drive/MyDrive/ACM Research/SAM2_boundingBox"

# List to store loaded images
images = []

# Load all PNG images from the folder
for filename in os.listdir(image_folder):
    if filename.endswith(".png"):
        image_path = os.path.join(image_folder, filename)
        image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
        images.append((filename, image))
        print(f"Loaded {filename}")

# Print the number of images loaded
print(f"Total images loaded: {len(images)}")


Loaded 097788d4-cb88-4457-8e71-0ca7a3da2216_processed.png
Loaded d5e9a92d-fb2f-416b-8731-4f7435d6c9c5_processed.png
Loaded d58012a2-a845-472d-8f49-336a3cc977c1_processed.png
Loaded d51472ae-a24e-4340-b468-54c883a12c41_processed.png
Loaded d55144f1-7b8b-424a-b460-5a7051abd301_processed.png
Loaded d508dfa3-6b86-49a1-816a-38bf8b1befbb_processed.png
Loaded d4a41252-bd49-47db-ae15-e6e489271b82_processed.png
Loaded d667579e-aa2d-461e-a869-4799ad466add_processed.png
Loaded d3a565b6-929e-4db1-8cc2-2b59e64860b5_processed.png
Loaded d094558c-2614-4d0b-b142-a9de0864f572_processed.png
Loaded d3fbcb3d-b39c-4c8b-a23d-ae786dc11bfe_processed.png
Loaded d184ed19-d81a-4168-9e45-21e1332390be_processed.png
Loaded d03b693d-8e4d-48ee-8cb3-8577680a55e4_processed.png
Loaded d0a2ca41-1f5b-475b-9cc0-b7efc4f19624_processed.png
Loaded d3bc87ed-6343-4196-b277-e94b01f1da40_processed.png
Loaded d230bf50-e18f-4b8a-bc79-678edb4e1261_processed.png
Loaded d18f4964-da90-4c37-bb7b-4ce1b2bbb6af_processed.png
Loaded d2e16b4

In [ ]:
# @title Entropy Modification Function (Gaussian Blur)
def modify_entropy(image, kernel_size):
    return cv2.GaussianBlur(image, (kernel_size, kernel_size), 0)


In [ ]:
# @title Entropy Filter Function with Disk Radius Testing
def apply_entropy_filter(image, disk_radius):
    # Define a local neighborhood for the entropy filter using a disk of variable radius
    neighborhood = disk(disk_radius)

    # Apply the entropy filter
    entropy_image = skimage_entropy(image, neighborhood)

    # Rescale the entropy image for better visualization (0-255 range)
    entropy_image_rescaled = img_as_ubyte(entropy_image / entropy_image.max())

    return entropy_image_rescaled


In [ ]:
# @title Compare SSIM and Entropy for Gaussian Blur and Entropy Filters
def compare_ssim_entropy(original_image, modified_image):
    # Calculate SSIM and entropy for both original and modified images
    ssim_value = ssim(original_image, modified_image)
    entropy_original = calculate_entropy(original_image)
    entropy_modified = calculate_entropy(modified_image)

    return ssim_value, entropy_original, entropy_modified

In [ ]:
# @title Create folder for saving results
save_folder = '/content/drive/MyDrive/ACM Research/Image_Comparison_Results'
if not os.path.exists(save_folder):
    os.makedirs(save_folder)

In [ ]:
# @title Save Side-by-Side Comparison Images
def save_comparison_image(original_image, modified_image, filename, param_type, param_value):
    # Create a side-by-side comparison image
    combined_image = np.hstack((original_image, modified_image))

    # Convert to PIL image and save
    combined_pil = Image.fromarray(combined_image)
    save_path = os.path.join(save_folder, f"{filename}_{param_type}_{param_value}.png")
    combined_pil.save(save_path)
    print(f"Saved comparison image: {save_path}")

In [ ]:
# @title Evaluate Best Parameters (Gaussian Blur and Entropy Filters)
def evaluate_parameters(image, filename):
    kernel_sizes = range(3, 20, 2)  # Odd kernel sizes for Gaussian blur
    disk_radii = range(1, 10, 2)    # Disk radii for entropy filter

    best_blur_ssim = 0
    best_blur_kernel = None
    best_filter_ssim = 0
    best_filter_radius = None

    results = []

    # Evaluate Gaussian blur
    for kernel_size in kernel_sizes:
        blur_image = modify_entropy(image, kernel_size)
        ssim_value, entropy_original, entropy_modified = compare_ssim_entropy(image, blur_image)

        results.append({
            'type': 'Gaussian Blur',
            'kernel_size': kernel_size,
            'ssim': ssim_value,
            'entropy_original': entropy_original,
            'entropy_modified': entropy_modified
        })

        # Save side-by-side comparison
        save_comparison_image(image, blur_image, filename, 'GaussianBlur_Kernel', kernel_size)

        if ssim_value > best_blur_ssim:
            best_blur_ssim = ssim_value
            best_blur_kernel = kernel_size

    # Evaluate Entropy Filter
    for disk_radius in disk_radii:
        filter_image = apply_entropy_filter(image, disk_radius)
        ssim_value, entropy_original, entropy_modified = compare_ssim_entropy(image, filter_image)

        results.append({
            'type': 'Entropy Filter',
            'disk_radius': disk_radius,
            'ssim': ssim_value,
            'entropy_original': entropy_original,
            'entropy_modified': entropy_modified
        })

        # Save side-by-side comparison
        save_comparison_image(image, filter_image, filename, 'EntropyFilter_Radius', disk_radius)

        if ssim_value > best_filter_ssim:
            best_filter_ssim = ssim_value
            best_filter_radius = disk_radius

    return {
        'filename': filename,
        'best_blur': {'kernel_size': best_blur_kernel, 'ssim': best_blur_ssim},
        'best_filter': {'disk_radius': best_filter_radius, 'ssim': best_filter_ssim},
        'results': results
    }


In [ ]:
# @title Run Evaluation on All Images and Summarize Results
all_results = []

for filename, image in images:
    result = evaluate_parameters(image, filename)
    all_results.append(result)
    print(f"Best parameters for {filename}:")
    print(f"  Best Gaussian Blur: Kernel Size = {result['best_blur']['kernel_size']}, SSIM = {result['best_blur']['ssim']}")
    print(f"  Best Entropy Filter: Disk Radius = {result['best_filter']['disk_radius']}, SSIM = {result['best_filter']['ssim']}")

# @title Plot Results for Gaussian Blur and Entropy Filter
def plot_results(result):
    blur_kernel_sizes = [r['kernel_size'] for r in result['results'] if r['type'] == 'Gaussian Blur']
    blur_ssim_values = [r['ssim'] for r in result['results'] if r['type'] == 'Gaussian Blur']
    blur_modified_entropy = [r['entropy_modified'] for r in result['results'] if r['type'] == 'Gaussian Blur']

    filter_disk_radii = [r['disk_radius'] for r in result['results'] if r['type'] == 'Entropy Filter']
    filter_ssim_values = [r['ssim'] for r in result['results'] if r['type'] == 'Entropy Filter']
    filter_modified_entropy = [r['entropy_modified'] for r in result['results'] if r['type'] == 'Entropy Filter']

    # Plot Gaussian Blur Results
    plt.figure(figsize=(10, 5))

    plt.subplot(1, 2, 1)
    plt.plot(blur_kernel_sizes, blur_ssim_values, marker='o', label='SSIM (Gaussian Blur)')
    plt.xlabel('Kernel Size')
    plt.ylabel('SSIM')
    plt.title(f'SSIM for {result["filename"]} (Gaussian Blur)')

    plt.subplot(1, 2, 2)
    plt.plot(blur_kernel_sizes, blur_modified_entropy, marker='o', color='orange', label='Modified Entropy (Gaussian Blur)')
    plt.axhline(blur_modified_entropy[0], color='red', linestyle='--', label='Original Entropy')
    plt.xlabel('Kernel Size')
    plt.ylabel('Entropy')
    plt.title(f'Entropy for {result["filename"]} (Gaussian Blur)')

    plt.legend()
    plt.tight_layout()
    plt.show()

    # Plot Entropy Filter Results
    plt.figure(figsize=(10, 5))

    plt.subplot(1, 2, 1)
    plt.plot(filter_disk_radii, filter_ssim_values, marker='o', label='SSIM (Entropy Filter)')
    plt.xlabel('Disk Radius')
    plt.ylabel('SSIM')
    plt.title(f'SSIM for {result["filename"]} (Entropy Filter)')

    plt.subplot(1, 2, 2)
    plt.plot(filter_disk_radii, filter_modified_entropy, marker='o', color='green', label='Modified Entropy (Entropy Filter)')
    plt.axhline(filter_modified_entropy[0], color='red', linestyle='--', label='Original Entropy')
    plt.xlabel('Disk Radius')
    plt.ylabel('Entropy')
    plt.title(f'Entropy for {result["filename"]} (Entropy Filter)')

    plt.legend()
    plt.tight_layout()
    plt.show()

Saved comparison image: /content/drive/MyDrive/ACM Research/Image_Comparison_Results/097788d4-cb88-4457-8e71-0ca7a3da2216_processed.png_GaussianBlur_Kernel_3.png
Saved comparison image: /content/drive/MyDrive/ACM Research/Image_Comparison_Results/097788d4-cb88-4457-8e71-0ca7a3da2216_processed.png_GaussianBlur_Kernel_5.png
Saved comparison image: /content/drive/MyDrive/ACM Research/Image_Comparison_Results/097788d4-cb88-4457-8e71-0ca7a3da2216_processed.png_GaussianBlur_Kernel_7.png
Saved comparison image: /content/drive/MyDrive/ACM Research/Image_Comparison_Results/097788d4-cb88-4457-8e71-0ca7a3da2216_processed.png_GaussianBlur_Kernel_9.png
Saved comparison image: /content/drive/MyDrive/ACM Research/Image_Comparison_Results/097788d4-cb88-4457-8e71-0ca7a3da2216_processed.png_GaussianBlur_Kernel_11.png
Saved comparison image: /content/drive/MyDrive/ACM Research/Image_Comparison_Results/097788d4-cb88-4457-8e71-0ca7a3da2216_processed.png_GaussianBlur_Kernel_13.png
Saved comparison image: /c

KeyboardInterrupt: 

In [ ]:
# @title Visualize Results for All Images
for result in all_results:
    plot_results(result)